In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive/training/

Mounted at /content/drive
'new 1.py'		       news-commentary-v9.fr-en.en
 news-commentary-v9.cs-en.cs   news-commentary-v9.fr-en.fr
 news-commentary-v9.cs-en.en   news-commentary-v9.ru-en.en
 news-commentary-v9.de-en.de   news-commentary-v9.ru-en.ru
 news-commentary-v9.de-en.en


In [46]:
with open('/content/drive/MyDrive/training/news-commentary-v9.de-en.en','rb') as f:
  train_en=f.readlines()
with open('/content/drive/MyDrive/training/news-commentary-v9.de-en.de','rb') as f:
  train_fr=f.readlines()
print(len(train_en),len(train_fr))
max=0
for j,i in enumerate(train_en+train_fr):
  if len(i.decode('utf-8').split())>max:
    max=len(i.decode('utf-8').split())
print(max)
MAX_LENGTH=max

201288 201288
193


In [47]:
%matplotlib inline
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [48]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [49]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):    
    s=s.lower().strip()
    s=s.decode('utf-8')
    s = unicodeToAscii(s)
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [50]:
pairs = [[normalizeString(i),normalizeString(j)] for i,j in zip(train_fr,train_en)]
len(pairs)

201288

In [ ]:
#MAX_LENGTH = 223
#no need to run this cell
def filterPair(p):
    return len(p[0].split(' ')) <= MAX_LENGTH and \
        len(p[1].split(' ')) <= MAX_LENGTH #and \
        #p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]
pairs = filterPairs(pairs)
#len(pairs)

In [52]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=5)
for i,j in kf.split(pairs):
  print(len(i),len(j))
 

161029 40258
161029 40258
161030 40257
161030 40257
161030 40257


In [53]:
input_lang = Lang('de')
output_lang = Lang('eng')
print("Read %s sentence pairs" % len(pairs))
#pairs = filterPairs(pairs)
#print("Trimmed to %s sentence pairs" % len(pairs))
#num_test = int(len(pairs)/5)
#print(num_test)
#l1=pairs[:num_test]
#l2=pairs[num_test:num_test*2]
#l3=pairs[num_test*2:num_test*3]
#l4=pairs[num_test*3:num_test*4]
#l5=pairs[num_test*4:]
def massage_test_data(test_pairs):
  set_test_eng = set([sent_eng for sent_eng, _ in test_pairs])

  test_pair_dict = {}
  for sent_eng, sent_fre in pairs:
      if sent_eng not in set_test_eng:
          continue 
      elif sent_eng not in test_pair_dict:
          test_pair_dict[sent_eng] = set([sent_fre])
      else:
          test_pair_dict[sent_eng].add(sent_fre)
  return [(sent_eng, list(test_pair_dict[sent_eng])) for sent_eng in test_pair_dict]

test_pairs = massage_test_data(pairs[161030:])
train_pairs= pairs[:161030]
print("Number of train pairs:", len(train_pairs))

Read 201287 sentence pairs
Number of train pairs: 161030


In [55]:
print("Counting words...")
for pair in train_pairs:
    input_lang.addSentence(pair[0])
    output_lang.addSentence(pair[1])
print(input_lang.name, input_lang.n_words)
print(output_lang.name, output_lang.n_words)


Counting words...
de 118724
eng 44328


In [56]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ') if word in lang.word2index]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def trainIters(encoder, decoder, n_iters,train_pairs,input_lang,output_lang, print_every=1000, plot_every=100, learning_rate=0.01):
    #train_pairs=train_pairs,input_lang=input_lang,output_lang=output_lang,
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    print('in train iters',len(train_pairs))
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(train_pairs))
                      for i in range(n_iters)]
    #print(training_pairs[0])
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    #showPlot(plot_losses)

In [57]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [58]:
from nltk.translate.bleu_score import corpus_bleu

def evaluateBleu(encoder, decoder, test_pairs):
    references, candidates = [], []    
    for sents_src, sents_tgt in test_pairs:      
      tgt_words    = [sent_tgt.split(' ') for sent_tgt in sents_tgt]      
      src_words, _ = evaluate(encoder, decoder, sents_src)      
      references.append(tgt_words)
      candidates.append(src_words)      
    score1 = corpus_bleu(references, candidates,weights=(1,0,0,0))
    score2 = corpus_bleu(references, candidates,weights=(0,1,0,0))
    score3 = corpus_bleu(references, candidates,weights=(0,0,1,0))
    #for individual BLEU's-1 2 & 3
    return score1,score2,score3

Training and Evaluating

**For BLEU 1,2,3 of language pairs on Q3 of tutorial 6 for Q2b iii part 4**

In [59]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
trainIters(encoder1, attn_decoder1, 5000,
             train_pairs=train_pairs,input_lang=input_lang,output_lang=output_lang, 
             print_every=1000)
b1,b2,b3=evaluateBleu(encoder1, attn_decoder1,test_pairs)
print('BLEUs :',b1,b2,b3)  

in train iters 161030
1m 53s (- 7m 34s) (1000 20%) 6.1739
3m 49s (- 5m 44s) (2000 40%) 5.7445
5m 45s (- 3m 50s) (3000 60%) 5.5384
7m 37s (- 1m 54s) (4000 80%) 5.4681
9m 32s (- 0m 0s) (5000 100%) 5.4956


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEUs : 0.15918576633265427 0.008245659719726232 0.5542767928803071
